# Chapter 36: Pytest Features

This notebook covers the essential features of pytest, Python's most popular testing framework. Since pytest fixtures, parametrize decorators, and markers require the pytest runner to execute, we demonstrate the concepts by showing the code patterns, explaining how they work, and using regular function calls to simulate behavior.

## Key Concepts
- **Fixtures**: Reusable setup/teardown functions that provide test data and resources
- **Parametrize**: Run the same test logic with multiple sets of inputs
- **Markers**: Tag tests for selective execution (skip, xfail, custom)
- **monkeypatch**: Temporarily modify objects, dictionaries, and environment variables
- **tmp_path**: Get a unique temporary directory for each test
- **pytest.raises**: Assert that code raises a specific exception
- **pytest.approx**: Compare floating-point numbers with tolerance

## Section 1: Understanding Fixtures

Fixtures are functions decorated with `@pytest.fixture` that provide reusable test data or resources. Tests declare dependencies on fixtures by accepting them as parameters. Here we simulate fixture behavior with regular functions.

In [ ]:
# In a real pytest file, you'd write:
#
# @pytest.fixture
# def sample_user() -> dict[str, str | int]:
#     return {"name": "Alice", "age": 30}
#
# def test_user_name(sample_user: dict[str, str | int]) -> None:
#     assert sample_user["name"] == "Alice"

# Simulating fixture behavior with a regular function
def sample_user_fixture() -> dict[str, str | int]:
    """Fixture that provides a sample user dictionary."""
    return {"name": "Alice", "age": 30}


# Simulate a test that uses the fixture
def test_user_name() -> None:
    user: dict[str, str | int] = sample_user_fixture()
    assert user["name"] == "Alice"
    assert isinstance(user["age"], int)
    print(f"User: {user}")
    print(f"Name check passed: {user['name']} == 'Alice'")
    print(f"Age is int: {isinstance(user['age'], int)}")


test_user_name()

In [ ]:
# Fixtures can have different scopes:
# - "function" (default): created fresh for each test
# - "class": shared across all tests in a class
# - "module": shared across all tests in a module
# - "session": shared across the entire test session

# Simulating a fixture with setup and teardown using yield
from typing import Generator


def database_connection_fixture() -> Generator[dict[str, str], None, None]:
    """Fixture with setup and teardown (yield fixture)."""
    # Setup phase
    connection: dict[str, str] = {"host": "localhost", "status": "connected"}
    print(f"SETUP: Opening connection to {connection['host']}")

    yield connection  # This is what the test receives

    # Teardown phase (runs after test completes)
    connection["status"] = "closed"
    print(f"TEARDOWN: Connection closed")


# Simulate the full fixture lifecycle
gen: Generator[dict[str, str], None, None] = database_connection_fixture()
conn: dict[str, str] = next(gen)  # Setup runs, we get the yielded value
print(f"TEST: Using connection with status '{conn['status']}'")
try:
    next(gen)  # Trigger teardown
except StopIteration:
    pass

## Section 2: Parametrize -- Multiple Inputs, One Test

The `@pytest.mark.parametrize` decorator runs a single test function with multiple sets of arguments. This eliminates repetitive test code. We simulate this pattern with a loop.

In [ ]:
# In a real pytest file:
#
# @pytest.mark.parametrize("value, expected", [
#     (1, 1), (2, 4), (3, 9), (4, 16), (0, 0),
# ])
# def test_square(value: int, expected: int) -> None:
#     assert value ** 2 == expected

# Simulating parametrize with a loop
test_cases: list[tuple[int, int]] = [
    (1, 1),
    (2, 4),
    (3, 9),
    (4, 16),
    (0, 0),
]

print("Parametrized square tests:")
for value, expected in test_cases:
    result: int = value ** 2
    passed: bool = result == expected
    status: str = "PASSED" if passed else "FAILED"
    print(f"  {value}^2 = {result}, expected {expected} -> {status}")
    assert result == expected

In [ ]:
# Parametrize with multiple parameters and IDs
# In pytest:
#
# @pytest.mark.parametrize("text, expected_words", [
#     ("hello world", 2),
#     ("", 0),
#     ("one", 1),
#     ("  spaces  between  ", 2),
# ], ids=["two_words", "empty", "single", "extra_spaces"])

def count_words(text: str) -> int:
    """Count non-empty words in a string."""
    return len(text.split()) if text.strip() else 0


word_count_cases: list[tuple[str, int, str]] = [
    ("hello world", 2, "two_words"),
    ("", 0, "empty"),
    ("one", 1, "single"),
    ("  spaces  between  ", 2, "extra_spaces"),
]

print("Parametrized word count tests:")
for text, expected_words, test_id in word_count_cases:
    result: int = count_words(text)
    passed: bool = result == expected_words
    status: str = "PASSED" if passed else "FAILED"
    print(f"  [{test_id}] count_words({text!r}) = {result}, expected {expected_words} -> {status}")
    assert result == expected_words

## Section 3: Markers -- Tagging and Controlling Tests

Markers let you tag tests for selective execution. Common built-in markers include `skip`, `skipif`, and `xfail`. Custom markers let you group tests by category.

In [ ]:
import sys

# In pytest:
#
# @pytest.mark.skip(reason="Not implemented yet")
# def test_future_feature() -> None: ...
#
# @pytest.mark.skipif(sys.platform == "win32", reason="Unix only")
# def test_unix_feature() -> None: ...
#
# @pytest.mark.xfail(reason="Known bug #123")
# def test_known_bug() -> None: ...

# Simulating marker behavior
def simulate_skip(reason: str) -> None:
    print(f"  SKIPPED: {reason}")


def simulate_skipif(condition: bool, reason: str) -> bool:
    if condition:
        print(f"  SKIPPED: {reason}")
        return True
    return False


def simulate_xfail(test_func: object, reason: str) -> None:
    try:
        if callable(test_func):
            test_func()
        print(f"  XPASS (unexpectedly passed): {reason}")
    except AssertionError:
        print(f"  XFAIL (expected failure): {reason}")


print("Marker demonstrations:")
print("\n@pytest.mark.skip:")
simulate_skip("Not implemented yet")

print("\n@pytest.mark.skipif:")
simulate_skipif(sys.platform != "win32", "Windows only test")

print("\n@pytest.mark.xfail:")
simulate_xfail(lambda: None, "Known bug #123 -- test passed unexpectedly")

In [ ]:
# Custom markers allow grouping tests by category
# In pytest.ini or pyproject.toml:
#   [tool.pytest.ini_options]
#   markers = [
#       "slow: marks tests as slow",
#       "integration: marks integration tests",
#   ]
#
# Usage:
#   @pytest.mark.slow
#   def test_large_dataset() -> None: ...
#
# Run only slow tests:  pytest -m slow
# Exclude slow tests:   pytest -m "not slow"

# Simulating custom markers with a registry
marker_registry: dict[str, list[str]] = {
    "slow": [],
    "integration": [],
    "unit": [],
}


def register_test(marker: str, test_name: str) -> None:
    marker_registry[marker].append(test_name)


register_test("unit", "test_add")
register_test("unit", "test_subtract")
register_test("integration", "test_database_query")
register_test("slow", "test_large_dataset")
register_test("slow", "test_performance_benchmark")

print("Custom marker registry:")
for marker, tests in marker_registry.items():
    print(f"  @pytest.mark.{marker}: {tests}")

# Filter by marker (simulating pytest -m)
selected_marker: str = "unit"
print(f"\nRunning with -m {selected_marker}: {marker_registry[selected_marker]}")

## Section 4: Monkeypatch -- Temporary Modifications

The `monkeypatch` fixture lets you temporarily modify attributes, dictionary items, and environment variables during a test. Changes are automatically reverted after the test.

In [ ]:
# In pytest, monkeypatch is a built-in fixture:
#
# def test_config(monkeypatch) -> None:
#     monkeypatch.setattr(Config, "debug", True)
#     assert Config.debug is True

# Simulating monkeypatch.setattr
class Config:
    debug: bool = False
    log_level: str = "INFO"


print(f"Before monkeypatch: Config.debug = {Config.debug}")
print(f"Before monkeypatch: Config.log_level = {Config.log_level}")

# Save originals, patch, then restore (what monkeypatch does internally)
original_debug: bool = Config.debug
original_level: str = Config.log_level

Config.debug = True
Config.log_level = "DEBUG"

print(f"\nDuring test: Config.debug = {Config.debug}")
print(f"During test: Config.log_level = {Config.log_level}")
assert Config.debug is True
assert Config.log_level == "DEBUG"

# Restore (monkeypatch does this automatically)
Config.debug = original_debug
Config.log_level = original_level

print(f"\nAfter test: Config.debug = {Config.debug}")
print(f"After test: Config.log_level = {Config.log_level}")

In [ ]:
import os

# monkeypatch.setenv / monkeypatch.delenv for environment variables
# In pytest:
#
# def test_env(monkeypatch) -> None:
#     monkeypatch.setenv("DATABASE_URL", "sqlite:///:memory:")
#     assert os.environ["DATABASE_URL"] == "sqlite:///:memory:"

# Simulating monkeypatch.setenv
env_key: str = "MY_TEST_VAR"
print(f"Before: {env_key!r} in os.environ = {env_key in os.environ}")

os.environ[env_key] = "test_value"
print(f"During test: os.environ[{env_key!r}] = {os.environ[env_key]!r}")
assert os.environ[env_key] == "test_value"

# Cleanup (monkeypatch handles this automatically)
del os.environ[env_key]
print(f"After test: {env_key!r} in os.environ = {env_key in os.environ}")

## Section 5: tmp_path -- Temporary Directories

The `tmp_path` fixture provides a `pathlib.Path` to a unique temporary directory for each test. Files created there are automatically cleaned up. This is ideal for testing file I/O.

In [ ]:
from pathlib import Path
import tempfile

# In pytest:
#
# def test_write_file(tmp_path: Path) -> None:
#     file = tmp_path / "test.txt"
#     file.write_text("hello")
#     assert file.read_text() == "hello"

# Simulating tmp_path with tempfile
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_path: Path = Path(tmp_dir)

    # Write a file
    test_file: Path = tmp_path / "test.txt"
    test_file.write_text("hello")

    # Read it back
    content: str = test_file.read_text()
    print(f"File path: {test_file}")
    print(f"File exists: {test_file.exists()}")
    print(f"Content: {content!r}")
    assert content == "hello"

    # Create subdirectories
    sub_dir: Path = tmp_path / "subdir"
    sub_dir.mkdir()
    nested_file: Path = sub_dir / "data.json"
    nested_file.write_text('{"key": "value"}')

    print(f"\nNested file: {nested_file}")
    print(f"Nested content: {nested_file.read_text()}")

# After the context manager, everything is cleaned up
print(f"\nTemp dir still exists: {Path(tmp_dir).exists()}")

In [ ]:
import tempfile
from pathlib import Path


# Practical example: testing a function that reads/writes files
def save_report(directory: Path, name: str, lines: list[str]) -> Path:
    """Save a report file and return the path."""
    file_path: Path = directory / f"{name}.txt"
    file_path.write_text("\n".join(lines))
    return file_path


def load_report(file_path: Path) -> list[str]:
    """Load a report file and return lines."""
    return file_path.read_text().splitlines()


# Test using a temporary directory (simulating tmp_path)
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_path: Path = Path(tmp_dir)

    report_lines: list[str] = ["Header", "Line 1", "Line 2"]
    saved_path: Path = save_report(tmp_path, "monthly", report_lines)

    print(f"Saved to: {saved_path.name}")
    print(f"File exists: {saved_path.exists()}")

    loaded: list[str] = load_report(saved_path)
    print(f"Loaded lines: {loaded}")
    assert loaded == report_lines
    print("Round-trip test PASSED")

## Section 6: pytest.raises -- Testing Exceptions

`pytest.raises` is a context manager that asserts a block of code raises a specific exception. It can also match against the exception message using a regex pattern.

In [ ]:
import re


# In pytest:
#
# def test_raises() -> None:
#     with pytest.raises(ValueError, match="invalid"):
#         raise ValueError("invalid literal")

# Simulating pytest.raises behavior
def assert_raises(
    exception_type: type[BaseException],
    callable_fn: object,
    match: str | None = None,
) -> None:
    """Simulate pytest.raises with optional message matching."""
    try:
        if callable(callable_fn):
            callable_fn()
        print(f"  FAILED: {exception_type.__name__} was not raised")
    except exception_type as e:
        if match and not re.search(match, str(e)):
            print(f"  FAILED: Message {str(e)!r} did not match {match!r}")
        else:
            print(f"  PASSED: Caught {exception_type.__name__}: {e}")
    except Exception as e:
        print(f"  FAILED: Expected {exception_type.__name__}, got {type(e).__name__}: {e}")


# Test 1: Basic exception check
print("Test: ValueError is raised")
assert_raises(ValueError, lambda: int("not_a_number"))

# Test 2: Exception with message matching
print("\nTest: ValueError with 'invalid' in message")
assert_raises(
    ValueError,
    lambda: (_ for _ in ()).throw(ValueError("invalid literal")),
    match="invalid",
)

# Test 3: ZeroDivisionError
print("\nTest: ZeroDivisionError on division by zero")
assert_raises(ZeroDivisionError, lambda: 1 / 0)

In [ ]:
# Using actual pytest.raises (it works in notebooks too)
import pytest


def validate_age(age: int) -> int:
    """Validate that age is a positive integer."""
    if age < 0:
        raise ValueError(f"Age cannot be negative: {age}")
    if age > 150:
        raise ValueError(f"Age is unrealistically high: {age}")
    return age


# Test valid input
result: int = validate_age(25)
print(f"validate_age(25) = {result}")

# Test negative age
with pytest.raises(ValueError, match="cannot be negative"):
    validate_age(-5)
print("Caught ValueError for negative age")

# Test too-high age
with pytest.raises(ValueError, match="unrealistically high"):
    validate_age(200)
print("Caught ValueError for too-high age")

# Access the exception info
with pytest.raises(ValueError) as exc_info:
    validate_age(-1)
print(f"\nException type: {type(exc_info.value).__name__}")
print(f"Exception message: {exc_info.value}")

## Section 7: pytest.approx -- Floating-Point Comparison

Floating-point arithmetic can produce tiny rounding errors. `pytest.approx` compares values with a configurable tolerance, avoiding brittle equality checks.

In [ ]:
import pytest

# The classic floating-point problem
result: float = 0.1 + 0.2
print(f"0.1 + 0.2 = {result}")
print(f"0.1 + 0.2 == 0.3: {result == 0.3}")

# pytest.approx handles this gracefully
print(f"0.1 + 0.2 == pytest.approx(0.3): {result == pytest.approx(0.3)}")
assert 0.1 + 0.2 == pytest.approx(0.3)

# Works with lists too
computed: list[float] = [0.1, 0.2, 0.1 + 0.2]
expected: list[float] = [0.1, 0.2, 0.3]
print(f"\nList comparison: {computed == pytest.approx(expected)}")
assert computed == pytest.approx(expected)

In [ ]:
import pytest
import math

# Custom tolerance with abs and rel parameters
# Default: rel=1e-6, abs=1e-12

# Absolute tolerance: values must be within abs of each other
assert 100.0 == pytest.approx(100.001, abs=0.01)
print(f"100.0 ~= 100.001 (abs=0.01): True")

# Relative tolerance: values must be within rel fraction of expected
assert 100.0 == pytest.approx(100.05, rel=0.001)
print(f"100.0 ~= 100.05 (rel=0.001): True")

# Practical example: testing math functions
angle: float = math.pi / 4
sin_val: float = math.sin(angle)
cos_val: float = math.cos(angle)
expected_val: float = math.sqrt(2) / 2

assert sin_val == pytest.approx(expected_val)
assert cos_val == pytest.approx(expected_val)
print(f"\nsin(pi/4) = {sin_val}")
print(f"cos(pi/4) = {cos_val}")
print(f"sqrt(2)/2 = {expected_val}")
print(f"All approximately equal: True")

# Dictionary values with approx
computed_stats: dict[str, float] = {"mean": 3.33333, "std": 1.41421}
expected_stats: dict[str, float] = {"mean": 10 / 3, "std": math.sqrt(2)}
assert computed_stats == pytest.approx(expected_stats, abs=1e-4)
print(f"\nDict approx comparison passed with abs=1e-4")

## Section 8: Putting It All Together

Here is a complete example showing how multiple pytest features would work together in a real test file.

In [ ]:
import tempfile
from pathlib import Path


# Module under test
class UserService:
    """Simple user service for demonstration."""

    def __init__(self, data_dir: Path) -> None:
        self.data_dir: Path = data_dir

    def save_user(self, name: str, email: str) -> Path:
        """Save user data to a file."""
        if not name:
            raise ValueError("Name cannot be empty")
        file_path: Path = self.data_dir / f"{name.lower()}.txt"
        file_path.write_text(f"{name}\n{email}")
        return file_path

    def load_user(self, name: str) -> dict[str, str]:
        """Load user data from a file."""
        file_path: Path = self.data_dir / f"{name.lower()}.txt"
        if not file_path.exists():
            raise FileNotFoundError(f"User {name!r} not found")
        lines: list[str] = file_path.read_text().splitlines()
        return {"name": lines[0], "email": lines[1]}


# --- Test suite (simulating pytest features) ---

with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_path: Path = Path(tmp_dir)
    service: UserService = UserService(tmp_path)

    # Test 1: Fixture-provided service + tmp_path for file I/O
    saved: Path = service.save_user("Alice", "alice@example.com")
    assert saved.exists()
    print("test_save_user: PASSED")

    # Test 2: Round-trip save/load
    user: dict[str, str] = service.load_user("Alice")
    assert user == {"name": "Alice", "email": "alice@example.com"}
    print("test_load_user: PASSED")

    # Test 3: pytest.raises for empty name
    try:
        service.save_user("", "no@email.com")
        print("test_empty_name: FAILED (no exception)")
    except ValueError as e:
        assert "cannot be empty" in str(e)
        print("test_empty_name_raises: PASSED")

    # Test 4: pytest.raises for missing user
    try:
        service.load_user("nonexistent")
        print("test_missing_user: FAILED (no exception)")
    except FileNotFoundError as e:
        assert "not found" in str(e)
        print("test_missing_user_raises: PASSED")

    # Test 5: Parametrize-style multiple users
    users: list[tuple[str, str]] = [
        ("Bob", "bob@test.com"),
        ("Carol", "carol@test.com"),
        ("Dave", "dave@test.com"),
    ]
    for name, email in users:
        service.save_user(name, email)
        loaded: dict[str, str] = service.load_user(name)
        assert loaded["name"] == name
        assert loaded["email"] == email
    print(f"test_parametrized_users ({len(users)} cases): PASSED")

## Summary

### Fixtures
- **`@pytest.fixture`**: Declare reusable setup functions; tests request them by parameter name
- **Yield fixtures**: Use `yield` to provide a value and run teardown code after the test
- **Scopes**: `function` (default), `class`, `module`, `session` control fixture lifetime

### Parametrize
- **`@pytest.mark.parametrize("args", [...])`**: Run one test with many input sets
- **`ids` parameter**: Give human-readable names to each test case

### Markers
- **`@pytest.mark.skip`**: Unconditionally skip a test
- **`@pytest.mark.skipif(condition)`**: Skip based on a condition (platform, version, etc.)
- **`@pytest.mark.xfail`**: Mark a test as expected to fail
- **Custom markers**: Tag tests for selective execution with `-m`

### Built-in Fixtures
- **`monkeypatch`**: Temporarily modify attributes (`setattr`), env vars (`setenv`), and dict items (`setitem`)
- **`tmp_path`**: Provides a unique `pathlib.Path` temporary directory per test

### Assertion Helpers
- **`pytest.raises(ExcType, match=pattern)`**: Assert code raises a specific exception with optional message matching
- **`pytest.approx(value, rel=1e-6, abs=1e-12)`**: Floating-point comparison with tolerance; works with scalars, lists, and dicts